In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers, losses
import seaborn as sns
sns.set_style('whitegrid')

path = 'G:/ImageColorization/landscape Images/color data'

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   rotation_range=25,
                                   brightness_range=[0.2,1.2])

train = train_datagen.flow_from_directory(path, target_size=(160,160),batch_size=7000,class_mode=None)

from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
import numpy as np
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128)
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)
def Colorizer(input_shape):
  encoder_input = keras.Input(shape=input_shape)
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(encoder_input)
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(x)
  x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=1)(x)   
  x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)(x)
  x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', strides=1)(x)
  x = layers.Conv2D(256, (5, 5), activation='relu', padding='same', strides=1)(x)
  x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)(x)
    
  encoder_output =  layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(x)

  x = layers.UpSampling2D((2, 2))(encoder_output)
  x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=1)(x)
  x = layers.UpSampling2D((2, 2))(encoder_output)
  x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=1)(x)
  x = layers.UpSampling2D((2, 2))(x)
  x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=1)(x)
  x = layers.Conv2D(2, (3, 3), activation='tanh', padding='same', strides=1)(x)
  decoder_output = layers.UpSampling2D((2, 2))(x)
  return keras.Model(encoder_input, decoder_output, name="decoder")

colorizer = Colorizer((160,160,1))
colorizer.summary()
import tensorflow as tf
def loss(y_true,y_pred) :
    l = tf.sqrt(tf.reduce_mean(tf.square(tf.abs(y_true - y_pred))))
    return l
colorizer.compile(optimizer='adam', loss=loss , metrics=['accuracy','mae','mse'])
hist=colorizer.fit(X,Y,validation_split=0.2, epochs=20)
                                                                                                                                                                                                                                                                        ``colorizer.save('./predictor.h5')
import matplotlib.pyplot as plt
ax = plt.subplot(label= True)
plt.plot(hist.history["loss"], label = "loss")
plt.plot(hist.history["val_loss"], label = "val_loss")
plt.title("RMSE")
plt.xlabel("epochs")
plt.ylabel("measure")
plt.legend(["rmse", "val_rmse"])
ax = plt.subplot(label= True)
plt.plot(hist.history["mse"], label = "mse")
plt.plot(hist.history["val_mse"], label = "val_mse")
plt.title("MSE")
plt.xlabel("epochs")
plt.ylabel("measure")
plt.legend(["mse", "val_mse"])
ax = plt.subplot(label= True)
plt.plot(hist.history["mae"], label = "mae")
plt.plot(hist.history["val_mae"], label = "val_mae")
plt.title("MAE")
plt.xlabel("epochs")
plt.ylabel("measure")
plt.legend(["mae", "val_mae"])

from skimage.transform import resize
import os
color_me = []
img = img_to_array(load_img('/content/drive/MyDrive/landscape Images/gray data/gray/6001.jpg'))
img = resize(img ,(160,160))
color_me.append(img)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
print(color_me.shape)

output = colorizer.predict(color_me)
output = output * 128

from skimage.io import imsave
# Output colorizations
for i in range(len(output)):
    result = np.zeros((160, 160, 3))
    result[:,:,0] = color_me[i][:,:,0]
    result[:,:,1:] = output[i]
    plt.figure(figsize=(5,5))
    plt.imshow(lab2rgb(result))
    plt.grid(False)
# train_pred = model.predict(train_inp)
# test_pred = model.predict(test_inp)
# train_random=random.randint(1,trainsize)
# test_random=random.randint(1,testsize)
# check=np.interp(train_pred, (train_pred.min(), train_pred.
# max()), (0,255))
# check1=np.interp(test_pred, (test_pred.min(), test_pred.max()),
# (0,255))
# l_channel=test_inp[20]*255
# a_channel=check1[20,:,:,0]
# b_channel=check1[20,:,:,1]
# transfer = cv2.merge([l_channel, a_channel, b_channel])
# transfer = cv2.cvtColor(transfer.astype("uint8"),
# cv2.COLOR_LAB2BGR)